# Phân tích chủ đề và tóm tắt bài báo (từ `merged.json`)

Notebook này giúp bạn:

- Nạp dữ liệu từ `merged.json` (gộp các file VNExpress)
- Tiền xử lý văn bản cơ bản (loại bỏ URL, HTML, ký tự thừa)
- Phân tích chủ đề bằng LDA (Latent Dirichlet Allocation)
- Gán chủ đề trội cho từng bài báo và xuất kết quả
- Tóm tắt văn bản theo bài, theo chủ đề hoặc theo từ khóa

Gợi ý:
- Thay đổi số chủ đề `NUM_TOPICS` để điều chỉnh độ chi tiết.
- Các bước dùng thuần `pandas` và `scikit-learn` để chạy được ngay mà không cần mô hình nặng.



In [ ]:
# Cài đặt/thư viện cần thiết (nếu thiếu, bỏ comment để cài)
# !pip install pandas scikit-learn nltk beautifulsoup4

import json
import os
import re
from typing import List, Tuple

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


DATA_PATH = "merged.json"  # thay đổi nếu cần
OUTPUT_ASSIGNED_TOPICS = "articles_with_topics.csv"

NUM_TOPICS = 10  # số chủ đề LDA
MAX_FEATURES = 30000  # số lượng từ tối đa cho vectorizer
MIN_DF = 3  # từ xuất hiện tối thiểu ở 3 văn bản
MAX_DF = 0.9  # loại bỏ từ xuất hiện ở >90% văn bản
TOP_WORDS = 15  # số từ nổi bật hiển thị mỗi chủ đề

RANDOM_STATE = 42



In [ ]:
def strip_html(text: str) -> str:
    if not isinstance(text, str):
        return ""
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text(separator=" ")


def basic_clean(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = strip_html(text)
    text = re.sub(r"https?://\S+", " ", text)
    text = re.sub(r"www\.\S+", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    return text


def load_articles(path: str) -> pd.DataFrame:
    # merged.json là mảng JSON các bài => list[dict]
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    # Chuẩn hóa các cột hay dùng: title, description, content/body, url, category, publishedAt
    rows = []
    for item in data:
        if not isinstance(item, dict):
            continue
        title = item.get("title") or item.get("headline") or item.get("name")
        desc = item.get("description") or item.get("summary")
        content = item.get("content") or item.get("body") or item.get("text")
        url = item.get("url") or item.get("link")
        category = item.get("category") or item.get("section") or item.get("cat")
        published = (
            item.get("publishedAt")
            or item.get("pubDate")
            or item.get("date")
            or item.get("created_at")
        )
        # Gộp văn bản để phân tích
        full_text = " ".join([
            str(title or ""),
            str(desc or ""),
            str(content or ""),
        ]).strip()
        rows.append({
            "title": title,
            "description": desc,
            "content": content,
            "url": url,
            "category": category,
            "published": published,
            "text": basic_clean(full_text),
        })
    df = pd.DataFrame(rows)
    # Loại các bản ghi rỗng văn bản
    df = df[df["text"].str.len() > 0].reset_index(drop=True)
    return df


df = load_articles(DATA_PATH)
print(df.shape)
df.head(3)


(7050, 7)


,title,description,content,url,category,published,text
0,Cao thủ duy nhất trong Kim Dung chết do võ côn...,Đây là một nhân vật đầy quyền lực và mưu mô tr...,Ông là giáo chủ đầy tham vọng của Nhật Nguyệt ...,https://vnexpress.net/crossword-giai-o-chu-o-c...,cuoi,None,Cao thủ duy nhất trong Kim Dung chết do võ côn...
1,Ảnh phòng tắm có điểm sai duy nhất nào?,Chỉ với 30 giây bạn có nhận ra điểm thiếu sót ...,Trong bức ảnh miêu tả căn phòng tắm với đầy đủ...,https://vnexpress.net/cau-do-iq-thu-tai-tinh-m...,cuoi,None,Ảnh phòng tắm có điểm sai duy nhất nào? Chỉ vớ...
2,Triều đại cuối cùng đóng đô tại hai kinh đô kh...,"Triều đại này tồn tại khoảng 24 năm, từng đóng...","Trước khi sụp đổ, triều đại này còn dự định đó...",https://vnexpress.net/crossword-giai-o-chu-o-c...,cuoi,None,Triều đại cuối cùng đóng đô tại hai kinh đô kh...


In [ ]:
# Vector hóa văn bản cho LDA (dùng Bag-of-Words)
vectorizer = CountVectorizer(
    max_features=MAX_FEATURES,
    min_df=MIN_DF,
    max_df=MAX_DF,
    stop_words=None,  # có thể cung cấp stopwords tiếng Việt nếu có
)
X_bow = vectorizer.fit_transform(df["text"])  # sparse matrix

lda = LatentDirichletAllocation(
    n_components=NUM_TOPICS,
    learning_method="batch",
    random_state=RANDOM_STATE,
)
W = lda.fit_transform(X_bow)  # document-topic matrix
H = lda.components_  # topic-word matrix

terms = np.array(vectorizer.get_feature_names_out())

def top_words_for_topic(topic_idx: int, n: int = TOP_WORDS) -> List[str]:
    idx = np.argsort(H[topic_idx])[::-1][:n]
    return terms[idx].tolist()

for k in range(NUM_TOPICS):
    print(f"Chủ đề {k}: ", ", ".join(top_words_for_topic(k)))


Chủ đề 0:  bệnh, thể, các, cơ, người, không, chất, khi, hoặc, là, cho, sĩ, thường, ăn, để
Chủ đề 1:  mỹ, một, là, này, ông, với, những, không, người, được, cho, khi, từ, đã, sẽ
Chủ đề 2:  bị, đã, không, cho, ông, các, được, người, công, ngày, khi, vụ, quan, án, nhân
Chủ đề 3:  các, công, với, cho, dụng, thể, hình, được, một, năng, người, động, ai, là, trên
Chủ đề 4:  trận, đội, bóng, thủ, đấu, với, là, khi, hai, sân, anh, thắng, giải, kết, một
Chủ đề 5:  tôi, không, người, một, là, con, khi, để, nhưng, làm, cho, anh, những, nhà, vì
Chủ đề 6:  khách, du, là, các, đến, với, được, từ, cho, nước, một, ăn, lịch, không, hàng
Chủ đề 7:  học, sinh, thi, các, điểm, năm, trường, đại, là, viên, với, được, thành, văn, nam
Chủ đề 8:  xe, đường, người, một, khi, trên, bị, với, máy, đi, ra, sau, tại, không, điện
Chủ đề 9:  công, các, doanh, chính, cho, đồng, với, năm, đầu, được, là, số, định, hàng, kinh


In [ ]:
# Gán chủ đề trội cho từng bài báo

dominant_topic = W.argmax(axis=1)
df_topics = df.copy()
df_topics["dominant_topic"] = dominant_topic

# Điểm (xác suất) của chủ đề trội để tham khảo
max_topic_score = W.max(axis=1)
df_topics["dominant_topic_score"] = max_topic_score

# Lưu ra CSV
cols = [
    "title", "url", "category", "published", "dominant_topic", "dominant_topic_score"
]
df_topics[cols].to_csv(OUTPUT_ASSIGNED_TOPICS, index=False)

print("Đã lưu:", OUTPUT_ASSIGNED_TOPICS)
df_topics.head(3)


Đã lưu: articles_with_topics.csv


,title,description,content,url,category,published,text,dominant_topic,dominant_topic_score
0,Cao thủ duy nhất trong Kim Dung chết do võ côn...,Đây là một nhân vật đầy quyền lực và mưu mô tr...,Ông là giáo chủ đầy tham vọng của Nhật Nguyệt ...,https://vnexpress.net/crossword-giai-o-chu-o-c...,cuoi,None,Cao thủ duy nhất trong Kim Dung chết do võ côn...,1,0.436070
1,Ảnh phòng tắm có điểm sai duy nhất nào?,Chỉ với 30 giây bạn có nhận ra điểm thiếu sót ...,Trong bức ảnh miêu tả căn phòng tắm với đầy đủ...,https://vnexpress.net/cau-do-iq-thu-tai-tinh-m...,cuoi,None,Ảnh phòng tắm có điểm sai duy nhất nào? Chỉ vớ...,5,0.505221
2,Triều đại cuối cùng đóng đô tại hai kinh đô kh...,"Triều đại này tồn tại khoảng 24 năm, từng đóng...","Trước khi sụp đổ, triều đại này còn dự định đó...",https://vnexpress.net/crossword-giai-o-chu-o-c...,cuoi,None,Triều đại cuối cùng đóng đô tại hai kinh đô kh...,1,0.475452


In [ ]:
# Tóm tắt trích xuất (extractive) đơn giản bằng TextRank cosine trên TF-IDF câu

sent_vectorizer = TfidfVectorizer(max_features=MAX_FEATURES)


def split_sentences(text: str) -> List[str]:
    # tách câu đơn giản theo dấu chấm/hỏi/cảm; có thể tinh chỉnh theo tiếng Việt
    sentences = re.split(r"(?<=[\.\?\!])\s+", text)
    sentences = [s.strip() for s in sentences if s and len(s.strip()) > 3]
    return sentences


def summarize_text(text: str, max_sentences: int = 3) -> str:
    sentences = split_sentences(text)
    if len(sentences) <= max_sentences:
        return " ".join(sentences)
    tfidf = sent_vectorizer.fit_transform(sentences)
    sim = cosine_similarity(tfidf)

    # TextRank: tính điểm PageRank đơn giản qua lặp power-iteration
    n = sim.shape[0]
    scores = np.ones(n) / n
    damping = 0.85
    for _ in range(30):
        new_scores = (1 - damping) / n + damping * sim.dot(scores) / np.maximum(sim.sum(axis=1), 1e-9)
        if np.allclose(new_scores, scores, atol=1e-6):
            break
        scores = new_scores
    top_idx = np.argsort(scores)[::-1][:max_sentences]
    top_idx = sorted(top_idx)  # giữ thứ tự gốc
    return " ".join(sentences[i] for i in top_idx)


# Ví dụ tóm tắt 5 bài đầu
examples = []
for i, row in df.head(5).iterrows():
    s = summarize_text(row["text"], max_sentences=3)
    examples.append({"title": row["title"], "summary": s})

pd.DataFrame(examples)


,title,summary
0,Cao thủ duy nhất trong Kim Dung chết do võ côn...,"Tuy nhiên, chính tuyệt kỹ này lại trở thành ng..."
1,Ảnh phòng tắm có điểm sai duy nhất nào?,"Mọi thứ dường như rất bình thường, nhưng có mộ..."
2,Triều đại cuối cùng đóng đô tại hai kinh đô kh...,Gợi ý:Người nắm quyền đầu tiên của triều đại n...
3,Xe máy chở gần trăm thùng nhựa cồng kềnh,> Thanh niên Hải Phòng lừng danh vì pha bay ng...
4,Người IQ cao sẽ biến phép tính sai thành đúng ...,Liệu bạn có thể tìm ra được bí mật đằng sau nh...


In [ ]:
# Tóm tắt theo chủ đề: nối text các bài trong một chủ đề rồi tóm tắt

def summarize_topic(topic_id: int, max_sentences: int = 5, max_articles: int = 200) -> str:
    subset = df_topics[df_topics["dominant_topic"] == topic_id]
    if subset.empty:
        return "(Không có bài phù hợp)"
    texts = subset["text"].head(max_articles).tolist()
    combined = ". ".join(texts)
    return summarize_text(combined, max_sentences=max_sentences)

# Ví dụ: tóm tắt 3 chủ đề đầu
for k in range(min(3, NUM_TOPICS)):
    print(f"\n==== TÓM TẮT CHỦ ĐỀ {k} ====")
    print(summarize_topic(k, max_sentences=5))



==== TÓM TẮT CHỦ ĐỀ 0 ====
Ngừa lão hóa miễn dịch, đẩy mạnh tiêm chủng ở người cao tuổi Chăm sóc sức khỏe chủ động và đầu tư tiêm chủng là cách giúp người cao tuổi phòng ngừa các bệnh mạn tính, truyền nhiễm. Tiêm chủng cho người lớn trung bình có thể mang lại lợi ích gấp 19 lần chi phí đầu tư ban đầu và lợi ích kinh tế, xã hội rộng hơn, cho phép người lớn tuổi duy trì sự năng động, khỏe mạnh lâu hơn. Các chuyên gia y tế khuyến cáo người dân cần thận trọng khi lựa chọn phẫu thuật ở nước ngoài, đặc biệt tại các cơ sở thiếu chứng nhận quốc tế hoặc đẩy nhanh quá trình phẫu thuật mà bỏ qua quy trình đánh giá sức khỏe và chuẩn bị tiền mê. Thổ Nhĩ Kỳ vẫn chưa có hệ thống giám sát chặt chẽ đối với các quảng cáo thẩm mỹ trên mạng xã hội, nơi các KOL thường đóng vai trò như "đại sứ thương hiệu" mà không đảm bảo tiêu chuẩn y khoa tương xứng. Thục Linh(TheoPeople)

==== TÓM TẮT CHỦ ĐỀ 1 ====
Cao thủ duy nhất trong Kim Dung chết do võ công mình tạo ra? Có một hệ thống đường dẫn nước đưa tới bốn ch

In [ ]:
# Tìm bài liên quan theo từ khóa và tóm tắt

def search_and_summarize(keyword: str, max_articles: int = 50, max_sentences: int = 5) -> Tuple[pd.DataFrame, str]:
    mask = df["text"].str.contains(re.escape(keyword), case=False, na=False)
    subset = df[mask].head(max_articles)
    if subset.empty:
        return subset, "(Không có bài phù hợp)"
    combined = ". ".join(subset["text"].tolist())
    return subset[["title", "url", "category", "published"]], summarize_text(combined, max_sentences=max_sentences)

# Ví dụ: tìm và tóm tắt theo từ khóa
results, summary = search_and_summarize("kinh doanh", max_articles=80, max_sentences=5)
summary, results.head(5)


('Bốn con giáp đếm tiền mỏi tay trong 3 ngày tới Trong 3 ngày tới, 4 con giáp được thần tài gõ cửa liên tục, tiền về không ngừng nghỉ, may mắn chạm ngõ từng việc nhỏ nhất. Xem ngay để xem bạn có phải là "con cưng của Thần may mắn" không nhé! >> 3 con giáp tài chính dồi dào vào cuối tuần * Tất nhiên những thông tin trong bài trắc nghiệm vui này chỉ mang tính chất tham khảo. Thành công thực sự trong cuộc sống luôn đòi hỏi sự nỗ lực không ngừng, trí tuệ, và những quyết định đúng đắn của mỗi cá nhân. Mai Nhật',
                                                  title  \
 35      Bốn con giáp đếm tiền mỏi tay trong 3 ngày tới   
 77    4 con giáp may mắn trong sự nghiệp không ai bằng   
 96   Bốn con giáp không lo thiếu tiền, sự nghiệp th...   
 129  Bốn con giáp tiền bạc đổ về dồn dập trong 2 th...   
 134  4 con giáp giàu lại càng giàu nhờ tài năng phi...   
 
                                                    url category published  
 35   https://vnexpress.net/van-may-12-con-giap-con-..